In [2]:
!pip install transformers

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Load model
print("Loading AI model...")
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Loading AI model...


In [ ]:
# Simple example (original code)
# text = "The small dog walked around the park happily with its owner and "
# print(f"\nStarting text: '{text}'")
# print("\nGenerating word by word...\n")

# # Generate 5 words, one at a time
# current_text = text
# for step in range(30):
#     print(f"--- Step {step + 1} ---")
#     print(f"Current: '{current_text}'")
    
#     # Encode current text
#     input_ids = tokenizer.encode(current_text, return_tensors="pt")
    
#     # Get predictions
#     with torch.no_grad():
#         outputs = model(input_ids)
#         predictions = outputs.logits
    
#     # Get the predictions for the NEXT token
#     next_token_logits = predictions[0, -1, :]
#     next_token_probs = torch.softmax(next_token_logits, dim=0)
    
#     # Get top 5 predictions
#     top_probs, top_indices = torch.topk(next_token_probs, 10)
    
#     print("Top 5 next word predictions:")
#     for i, (prob, idx) in enumerate(zip(top_probs, top_indices)):
#         word = tokenizer.decode([idx])
#         print(f"  {i+1}. '{word}' ({prob.item()*100:.1f}% confident)")
    
#     # Top K pred
#     # top_probs = top_probs / top_probs.sum()
#     # sampled_index = torch.multinomial(top_probs, 1)
#     # next_token_id = top_indices[sampled_index]
#     # next_word = tokenizer.decode(next_token_id)
#     # current_text += next_word

#     # Use the most likely word (greedy)
#     next_token_id = top_indices[0]
#     next_word = tokenizer.decode([next_token_id])
#     current_text += next_word
    
#     print(f"✓ Chosen: '{next_word}'")
#     print(f"New text: '{current_text}'\n")

# print(f"\nFinal generated text: '{current_text}'")

In [15]:
import re
import random


def normalize_text(raw_text):
    cleaned = re.sub(r"[^a-zA-Z\s\.]", "", raw_text)
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    return cleaned


def build_poem():
    starters = [
        "The moon is bright and",
    ]

    seed_text = random.choice(starters)

    generated = model.generate(
        tokenizer.encode(seed_text, return_tensors="pt"),
        max_length=140,
        do_sample=True,
        temperature=0.7,
        top_k=30,
        top_p=0.85,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(generated[0], skip_special_tokens=True)
    decoded = normalize_text(decoded)

    words = decoded.split()

    if len(words) < 45:
        return build_poem()

    poem_lines = []
    current_line = []
    index = 0

    while len(poem_lines) < 4 and index < len(words):
        word = words[index]
        current_line.append(word)
        index += 1

        word_count = len(current_line)
        ends_sentence = word.endswith(".")

        if word_count >= 5 and ends_sentence:
            poem_lines.append(" ".join(current_line))
            current_line = []
            continue

        if word_count == 7:
            poem_lines.append(" ".join(current_line))
            current_line = []

    if len(poem_lines) < 4:
        return build_poem()

    last_words = poem_lines[-1].split()
    bad_endings = ["the", "a", "an", "and", "or", "but", "we", "to", "in"]

    if last_words[-1].lower() in bad_endings:
        continuation = model.generate(
            tokenizer.encode(poem_lines[-1], return_tensors="pt"),
            max_length=20,
            do_sample=True,
            temperature=0.7,
            top_k=30,
            top_p=0.85,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )

        extra = tokenizer.decode(continuation[0], skip_special_tokens=True)
        extra = normalize_text(extra)
        extra_words = extra.split()[len(last_words):]

        for w in extra_words:
            last_words.append(w)
            if w.endswith(".") and len(last_words) >= 5:
                break

        poem_lines[-1] = " ".join(last_words[:7])

    return "\n".join(poem_lines)


print("\nGenerated poem:\n")
print(build_poem())


Generated poem:

The moon is bright and the sun
shines on it but there are still
some moons that arent in sight.
Ive been a little concerned about this
